In [3]:
from transformers import AutoTokenizer
import torch

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B")

In [8]:
prompt_strs = [
    "This is the first",
    "This is a fun assignment",
    "Internally, most modern tokenizers use PreTrainedTokenizerFast (Rust-backed, fast)."
]

output_strs = [
    "time I start coding directly",
    "so far.",
    "This is pretty convenient"
]




In [25]:
prompt_strs_tokenized = tokenizer(prompt_strs, padding=True, return_tensors="pt")
mask_prompts = prompt_strs_tokenized["attention_mask"]
print(prompt_strs_tokenized["input_ids"])

tensor([[[  1986,    374,    264,  ..., 151643, 151643, 151643],
         [  1986,    374,    264,  ..., 151643, 151643, 151643],
         [  1986,    374,    264,  ..., 151643, 151643, 151643],
         ...,
         [  1986,    374,    279,  ..., 151643, 151643, 151643],
         [  1986,    374,    279,  ..., 151643, 151643, 151643],
         [  1986,    374,    279,  ..., 151643, 151643, 151643]],

        [[  1986,    374,    264,  ..., 151643, 151643, 151643],
         [  1986,    374,    264,  ..., 151643, 151643, 151643],
         [  1986,    374,    264,  ..., 151643, 151643, 151643],
         ...,
         [  1986,    374,    279,  ..., 151643, 151643, 151643],
         [  1986,    374,    279,  ..., 151643, 151643, 151643],
         [  1986,    374,    279,  ..., 151643, 151643, 151643]],

        [[  1986,    374,    264,  ..., 151643, 151643, 151643],
         [  1986,    374,    264,  ..., 151643, 151643, 151643],
         [  1986,    374,    264,  ..., 151643, 151643, 15

In [22]:
output_strs_tokenized = tokenizer(output_strs, padding=True, return_tensors="pt")
output_strs_tokenized["input_ids"]

tensor([[  1678,    358,   1191,  10822,   5961],
        [   704,   3041,     13, 151643, 151643],
        [  1986,    374,   5020,  16710, 151643]])

In [20]:

mask_prompts = torch.where(mask_prompts == 1, torch.tensor(0), mask_prompts)
response_mask = torch.cat((mask_prompts, output_strs_tokenized["attention_mask"]), dim=-1)


output_input_ids = output_strs_tokenized["input_ids"]
prompt_input_ids = prompt_strs_tokenized["input_ids"]
input_ids = torch.cat((prompt_input_ids, output_input_ids[:, :-1]), dim=-1)
labels = torch.cat((prompt_input_ids[:, 1:], output_input_ids), dim=-1)
print("input_ids:", input_ids)
print("\n labels:", labels)
print("\n response_mask:", response_mask)

input_ids: tensor([[  1986,    374,    279,   1156, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643,   1678,    358,   1191,  10822],
        [  1986,    374,    264,   2464,  16319, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643,    704,   3041,     13, 151643],
        [ 67916,    745,     11,   1429,   6481,   3950,  12230,    990,   4968,
           1282,   2627,  37434,  32174,    320,     49,    590,  45028,     11,
           4937,    568,   1986,    374,   5020,  16710]])

 labels: tensor([[   374,    279,   1156, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643,   1678,    358,   1191,  10822,   5961],
        [   374,    264,   2464,  16319, 151643, 151643, 151643, 151643, 151643,
         151643, 

In [ ]:
{
    "input_ids": input_ids,
    "labels": labels,
    "response_mask": response_mask
}

In [30]:
enc_prompt = tokenizer(prompt_strs, )
enc_output = tokenizer(output_strs)

merged_input_ids = [a + b for a, b in zip(enc_prompt["input_ids"], enc_output["input_ids"])]
merged_mask = [[0] * len(a) + b for a, b in zip(enc_prompt["attention_mask"],enc_output["attention_mask"])]

print(merged_input_ids)
merged_mask

[[1986, 374, 279, 1156, 1678, 358, 1191, 10822, 5961], [1986, 374, 264, 2464, 16319, 704, 3041, 13], [67916, 745, 11, 1429, 6481, 3950, 12230, 990, 4968, 1282, 2627, 37434, 32174, 320, 49, 590, 45028, 11, 4937, 568, 1986, 374, 5020, 16710]]


[[0, 0, 0, 0, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]

In [38]:
padded_tensor = tokenizer.pad(
    {"input_ids": merged_input_ids, "attention_mask": merged_mask},
    padding=True,
    return_tensors="pt"
)
padded_tensor

{'input_ids': tensor([[  1986,    374,    279,   1156,   1678,    358,   1191,  10822,   5961,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643],
        [  1986,    374,    264,   2464,  16319,    704,   3041,     13, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643],
        [ 67916,    745,     11,   1429,   6481,   3950,  12230,    990,   4968,
           1282,   2627,  37434,  32174,    320,     49,    590,  45028,     11,
           4937,    568,   1986,    374,   5020,  16710]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [39]:
merged_input_ids

[[1986, 374, 279, 1156, 1678, 358, 1191, 10822, 5961],
 [1986, 374, 264, 2464, 16319, 704, 3041, 13],
 [67916,
  745,
  11,
  1429,
  6481,
  3950,
  12230,
  990,
  4968,
  1282,
  2627,
  37434,
  32174,
  320,
  49,
  590,
  45028,
  11,
  4937,
  568,
  1986,
  374,
  5020,
  16710]]

In [40]:
merged_input_ids[0][:-1]

[1986, 374, 279, 1156, 1678, 358, 1191, 10822]